# An adaptation of Nicholas' notebook (00.0_tks_rv_target_selection.ipynb)

In [1]:
# System
import os

# Analysis
import numpy as np
import pandas as pd 
import astropy as ap
from scipy.stats import binned_statistic_dd
from scipy.stats import linregress
from astroquery.mast import Catalogs
import mr_forecast as mr

# Plotting
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["font.family"] = "serif"
rcParams["font.serif"] = "Times New Roman"
%matplotlib inline
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = [r'\usepackage{amsmath} \usepackage{bm} \usepackage{physics}']
%config InlineBackend.figure_format = 'retina' # For high quality figures

# io
from astropy.io import ascii

# Development
%load_ext autoreload
%autoreload 2

### Note re: mr_forecast package 
*Note adapted from Nicholas' original notebook.*

(Nicholas) replaced the [Chen & Kipping](https://arxiv.org/pdf/1603.08614.pdf) function with the strictly analytical relationship implemented in [Louie et al. 2018](https://arxiv.org/pdf/1711.02098.pdf), which runs more quickly and for our purposes is sufficient. **EXCEPT for planets with $R > R_J$**.

## Notebook purpose:
Select TESS planet candidates for RV follow-up with Keck, so that we can constrain their masses in preparation for JWST observations.

In [2]:
# The local directory where the toi csv files are located.
toi_dir = 'toi_lists'

In [3]:
# The specific filename to use
toi_list_fname = os.path.join(toi_dir, 'toi+-2019-10-18.csv')

In [4]:
# Read in the data
toi_df = pd.read_csv(toi_list_fname, comment='#')

In [5]:
toi_df.head() # Examine the first few rows to make sure it was read in correctly

,Source Pipeline,TIC,Full TOI ID,TOI Disposition,TIC Right Ascension,TIC Declination,TMag Value,TMag Uncertainty,Orbital Epoch Value,Orbital Epoch Uncertainty,...,Centroid Offset,TFOP Master,TFOP SG1a,TFOP SG1b,TFOP SG2,TFOP SG3,TFOP SG4,TFOP SG5,Alerted,Edited
0,spoc,144065872,105.01,KP,337.457199,-48.003087,9.480,0.020,1326.505990,0.000156,...,False,5,5,5,5,5,5,5,2018-09-05T18:49:20+00:00,2019-10-17 00:41:47.471372+00:00
1,qlp,147203645,166.01,PC,323.195894,-44.017794,10.696,0.018,1326.384240,0.002100,...,False,3,4,3,3,4,4,4,2018-09-05T18:52:24+00:00,2019-10-17 00:41:46.780732+00:00
2,spoc,231670397,104.01,KP,319.949611,-58.148876,9.848,0.018,1327.673462,0.000842,...,False,5,5,5,5,5,5,5,2018-09-05T18:49:20+00:00,2019-10-17 00:41:44.380089+00:00
3,qlp,271893367,150.01,PC,112.965695,-73.606125,10.865,0.019,1326.278380,0.001390,...,False,3,4,3,3,4,4,4,2018-09-05T18:34:59+00:00,2019-10-18 21:38:54.240261+00:00
4,qlp,260985861,149.01,PC,76.635855,-80.768181,10.140,0.018,1327.402260,0.002000,...,False,1,4,1,1,4,4,4,2018-09-05T18:35:00+00:00,2019-10-18 21:39:04.252414+00:00


In [8]:
toi_df.columns

Index(['Source Pipeline', 'TIC', 'Full TOI ID', 'TOI Disposition',
       'TIC Right Ascension', 'TIC Declination', 'TMag Value',
       'TMag Uncertainty', 'Orbital Epoch Value', 'Orbital Epoch Uncertainty',
       'Orbital Period Value', 'Orbital Period Uncertainty',
       'Transit Duration (hours) Value',
       'Transit Duration (hours) Uncertainty', 'Transit Depth Value',
       'Transit Depth Uncertainty', 'Sectors', 'Public Comment',
       'Surface Gravity Value', 'Surface Gravity Uncertainty', 'Signal ID',
       'Star Radius Value', 'Star Radius Uncertainty', 'Planet Radius Value',
       'Planet Radius Uncertainty', 'Planet Equilibrium Temperature (K) Value',
       'Effective Temperature Value', 'Effective Temperature Uncertainty',
       'Effective Stellar Flux Value', 'Signal-to-noise', 'Centroid Offset',
       'TFOP Master', 'TFOP SG1a', 'TFOP SG1b', 'TFOP SG2', 'TFOP SG3',
       'TFOP SG4', 'TFOP SG5', 'Alerted', 'Edited'],
      dtype='object')

In [ ]:
## 